In [1]:
import os
import pandas as pd
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
import re
import nltk
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from gensim.models.doc2vec import TaggedDocument
import pickle
import json

In [ ]:
sequence = ''
with open('./data/uniprot/frequent_kmers.json', 'r') as fp:
    kmers = json.load(fp)
dictionary = kmers
import re
from collections import Counter

tokenized_data = {}

def word_prob(word): 
    if word in dictionary:
        return dictionary[word] / total
    else:
        return 0
    
    
    
def words(text): 
    return re.findall('[A-Z]+', text) 


max_word_length = max(map(len, dictionary))
total = float(sum(dictionary.values()))
print(max_word_length)

def viterbi_segment(text):
    probs, lasts = [1.0], [0]
    for i in range(1, len(text) + 1):
        prob_k, k = max((probs[j] * word_prob(text[j:i]), j)
        for j in range(max(0, i - max_word_length), i))
        probs.append(prob_k)
        lasts.append(k)
    words = []
    i = len(text)
    while 0 < i:
        words.append(text[lasts[i]:i])
        i = lasts[i]
    words.reverse()
    return words, probs[-1]

ctr = 0
flag = False
for line in tqdm(open('./data/Uniprot/uniprot_sprot.dat')):
    ctr += 1
#     print(line)
    if line.startswith('AC'):
        acc_id = line[2:].strip(' ')[:-2]
#         print(acc_id, '\n')
#     if ctr % 400 == 0:
#         break
    if flag:
        if line.startswith('/'):
            flag = False
#             print(sequence, '\n\n\n')
            tokenized_data[acc_id] = viterbi_segment(sequence)[0]
            sequence = ''
            continue
        sequence = sequence + ''.join(line[:-1].split(' '))
    if line.startswith('SQ'):
        flag = True

In [2]:
import pickle 
with open('./data/Uniprot/sequences/word2grayscale.pkl', 'rb') as pkl:
    word2grayscale = pickle.load(pkl)
with open('./data/Uniprot/sequences/tokenized_data.pkl', 'rb') as pkl:
    tokenized_data = pickle.load(pkl)

In [3]:
from tqdm.notebook import tqdm
import numpy as np
import math

ctr = 0
# !mkdir protein_images
DIR = './data/Uniprot/'

vec_size=222
from PIL import Image
for id in tqdm(list(tokenized_data.keys())):
    seq = tokenized_data[id]
    image = []
    for word in seq:
        try:
            image.append(word2grayscale[word])
        except:
            image.append(np.zeros(vec_size)) # unknown word
    image = np.array(image)
    # print(image.shape)
#     if vec_size > len(seq):
#         padLen = vec_size - len(seq)
#         topPad = int(padLen/2)
#         bottomPad = math.ceil(padLen/2)
#         top = np.zeros((topPad, vec_size))
#         bottom = np.zeros((bottomPad, vec_size))
#         image = np.vstack((top, image, bottom))
#     elif vec_size < len(seq):
#         padLen = len(seq) - vec_size
#         leftPad = int(padLen/2)
#         rightPad = math.ceil(padLen/2)
#         left = np.zeros((len(seq), leftPad))
#         right = np.zeros((len(seq), rightPad))
#         image = np.hstack((left, image, right))
    img = Image.fromarray(image.astype(np.uint8))
    img = img.resize((128 ,128), Image.ANTIALIAS)
    ctr += 1
    fid = id.split(';')[0]
    try:
        if ctr < 56300:
            img.save(os.path.join(DIR, 'protein_images/test/proteins/' + fid + '.png'), optimize=True, quality=100)
        elif ctr < 2*56300:
            img.save(os.path.join(DIR, 'protein_images/valid/proteins/' + fid + '.png'), optimize=True, quality=100)
        else:
            img.save(os.path.join(DIR, 'protein_images/train/proteins/' + fid + '.png'), optimize=True, quality=100)
    except:
        print('[SKIP]')
        continue
#     img.save('./data/Uniprot/protein_images/' + id + '-9.png', optimize=True, quality=100)
#     break

In [5]:
import os
train_dir = os.path.join(DIR, 'protein_images/train')
files = os.listdir(os.path.join(train_dir, 'proteins'))
files[:4]
import shutil
ctr = 1
fid = 0
try:
    os.mkdir(os.path.join(train_dir, str(fid)))
    os.mkdir(os.path.join(train_dir, str(fid), 'proteins'))
except:
    pass

for i in tqdm(range(len(files))):
    shutil.move(os.path.join(train_dir, 'proteins', files[i]), os.path.join(train_dir, str(fid), 'proteins', files[i]))
    ctr += 1
    if ctr % 8192 == 0:
        fid += 1
        os.mkdir(os.path.join(train_dir, str(fid)))
        os.mkdir(os.path.join(train_dir, str(fid), 'proteins'))

In [6]:
import os
train_dir = os.path.join(DIR, 'protein_images/test')
files = os.listdir(os.path.join(train_dir, 'proteins'))
files[:4]
import shutil
ctr = 1
fid = 0
try:
    os.mkdir(os.path.join(train_dir, str(fid)))
    os.mkdir(os.path.join(train_dir, str(fid), 'proteins'))
except:
    pass

for i in tqdm(range(len(files))):
    shutil.move(os.path.join(train_dir, 'proteins', files[i]), os.path.join(train_dir, str(fid), 'proteins', files[i]))
    ctr += 1
    if ctr % 8192 == 0:
        fid += 1
        os.mkdir(os.path.join(train_dir, str(fid)))
        os.mkdir(os.path.join(train_dir, str(fid), 'proteins'))

In [7]:
import os
train_dir = os.path.join(DIR, 'protein_images/valid')
files = os.listdir(os.path.join(train_dir, 'proteins'))
files[:4]
import shutil
ctr = 1
fid = 0
try:
    os.mkdir(os.path.join(train_dir, str(fid)))
    os.mkdir(os.path.join(train_dir, str(fid), 'proteins'))
except:
    pass

for i in tqdm(range(len(files))):
    shutil.move(os.path.join(train_dir, 'proteins', files[i]), os.path.join(train_dir, str(fid), 'proteins', files[i]))
    ctr += 1
    if ctr % 8192 == 0:
        fid += 1
        os.mkdir(os.path.join(train_dir, str(fid)))
        os.mkdir(os.path.join(train_dir, str(fid), 'proteins'))

In [25]:
import os
train_dir = os.path.join(DIR, 'protein_images1/train')
dirs = os.listdir(os.path.join(train_dir))
dirs[:4]
import shutil

for j in tqdm(range(len(dirs))):
    files = os.listdir(os.path.join(train_dir, dirs[j]))
    os.mkdir(os.path.join(train_dir, dirs[j], 'proteins'))
    for i in range(len(files)):
        shutil.move(os.path.join(train_dir, dirs[j], files[i]), os.path.join(train_dir, dirs[j], 'proteins', files[i]))

In [ ]:
import pickle
with open('./data/Uniprot/sequences/tokenized_data.pkl', 'rb') as pkl:
    tokenized_data = pickle.load(pkl)
type(tokenized_data)

In [ ]:
df = []
DIR = './data/uniprot/annotations/'

def cleanText(text):
    text = BeautifulSoup(text, "lxml").text
    text = re.sub(r'\n', r' ', text)
    return text

def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            tokens.append(word.lower())
    return tokens


for file in tqdm_notebook(os.listdir(DIR)):
    df = pd.read_csv(DIR + file)
    df = df[['annotation', 'tag']]
    print(f'[CLEANING-{file}]')
    df['annotation'] = df['annotation'].apply(cleanText)
    print('[SPLITTING]')
    train, test = train_test_split(df, test_size=0.3, random_state=42)
    print('[TOKENIZING-{train_data}]')
    train_tagged = train.apply(lambda r: TaggedDocument(words=tokenize_text(r['annotation']), tags=[r.tag]), axis=1)
    print('[TOKENIZING-{test_data}]')
    test_tagged = test.apply(lambda r: TaggedDocument(words=tokenize_text(r['annotation']), tags=[r.tag]), axis=1)
    print(f'[DUMPING]....[SESSION-{file}]')
    train_tagged.to_pickle(DIR + 'train1/' + file[:-4] + '.pkl')
    test_tagged.to_pickle(DIR + 'test1/' + file[:-4] + '.pkl')
    print('[DUMPED]')

In [ ]:
DIR = './data/uniprot/annotations/train/'
df = []
for file in tqdm_notebook(os.listdir(DIR)):
    if len(df) == 0:
        print('[INITIALIZING]')
        df = pd.read_pickle(DIR + file)
        print(f'[INITIALIZED-{file}]')
    else:
        temp = pd.read_pickle(DIR + file)
        print(f'[APPENDING-{file}]')
        df = pd.concat([df, temp])
        del temp
df.to_pickle(DIR + 'train.pkl')

In [ ]:
DIR = './data/uniprot/annotations/test/'
df = []
for file in tqdm_notebook(os.listdir(DIR)):
    if len(df) == 0:
        print('[INITIALIZING]')
        df = pd.read_pickle(DIR + file)
        print(f'[INITIALIZED-{file}]')
    else:
        temp = pd.read_pickle(DIR + file)
        print(f'[APPENDING-{file}]')
        df = pd.concat([df, temp])
        del temp
df.to_pickle(DIR + 'test.pkl')

In [ ]:
df = []
DIR = './data/uniprot/unknown_annotations/'

def cleanText(text):
    text = BeautifulSoup(text, "lxml").text
    text = re.sub(r'\n', r' ', text)
    return text

def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            tokens.append(word.lower())
    return tokens

df = []
for file in tqdm_notebook(os.listdir(DIR)):
    if len(df) == 0:
        df = pd.read_csv(DIR + file)
    else:
        temp = pd.read_csv(DIR + file)
        df = pd.concat([df, temp])
df = df[['annotation', 'tag']]
print(f'[CLEANING-{file}]')
df['annotation'] = df['annotation'].apply(cleanText)
print('[TOKENIZING-{df}]')
tagged = df.apply(lambda r: TaggedDocument(words=tokenize_text(r['annotation']), tags=[r.tag]), axis=1)
print(f'[DUMPING]....[SESSION-{file}]')
tagged.to_pickle(DIR  + 'unkown.pkl')
print('[DUMPED]')

In [ ]:
df = pd.read_csv('./data/uniprot/sequences/sequences_1.csv')
df.head()

In [ ]:
ctr = 0
for row in df.iterrows():
    print(row[1]['tag'])
    break

In [ ]:
df = []
DIR = './data/uniprot/sequences/'

def cleanText(text):
    text = BeautifulSoup(text, "lxml").text
    text = re.sub(r'\n', r' ', text)
    return text

def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            tokens.append(word.lower())
    return tokens

df = []
for file in tqdm_notebook(os.listdir(DIR)):
    if file.endswith('.csv'):
        if len(df) == 0:
            df = pd.read_csv(DIR + file)
        else:
            temp = pd.read_csv(DIR + file)
            df = pd.concat([df, temp])
df = df.groupby('tag').filter(lambda x : (x['tag'].count()>=1000).any())
df.head()

In [ ]:
print(df['tag'].nunique(), len(df))

In [ ]:
df = df[['sequence', 'tag']]
print(f'[CLEANING-{file}]')
df['sequence'] = df['sequence'].apply(cleanText)
print('[SPLITTING]')
train, test = train_test_split(df, test_size=0.3, random_state=42)
print('[TOKENIZING-{train_data}]')
train_tagged = train.apply(lambda r: TaggedDocument(words=tokenize_text(r['sequence']), tags=[r.tag]), axis=1)
print('[TOKENIZING-{test_data}]')
test_tagged = test.apply(lambda r: TaggedDocument(words=tokenize_text(r['sequence']), tags=[r.tag]), axis=1)
print(f'[DUMPING]....[SESSION-{file}]')
train_tagged.to_pickle(DIR + 'new/train/train.pkl')
test_tagged.to_pickle(DIR + 'new/test/test.pkl')
print('[DUMPED]')

In [ ]:
df = []
DIR = './data/uniprot/unknown_sequences/'

def cleanText(text):
    text = BeautifulSoup(text, "lxml").text
    text = re.sub(r'\n', r' ', text)
    return text

def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            tokens.append(word.lower())
    return tokens

df = []
for file in tqdm_notebook(os.listdir(DIR)):
    if len(df) == 0:
        df = pd.read_csv(DIR + file)
    else:
        temp = pd.read_csv(DIR + file)
        df = pd.concat([df, temp])
df = df[['sequence', 'tag']]
print(f'[CLEANING-{file}]')
df['sequence'] = df['sequence'].apply(cleanText)
print('[TOKENIZING-{df}]')
tagged = df.apply(lambda r: TaggedDocument(words=tokenize_text(r['sequence']), tags=[r.tag]), axis=1)
print(f'[DUMPING]....[SESSION-{file}]')
tagged.to_pickle(DIR  + 'unkown.pkl')
print('[DUMPED]')